In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
import pandas as pd
import ast

### Select data

In [6]:
df3=pd.read_csv('Results/happy_100images_H0.csv', sep=',')[0:100]
df4=pd.read_csv('Results/angry_100images_H0.csv', sep=',')[0:100]

### Select Train/Test Split

In [12]:
Train = 80
Test = 100-train

### Fit SVM

In [13]:
## Get data in correct shape

x2 = []

for i in range(Train):
    c1_temp = df3["content 1"][i]
    c1 = [float(x) for x in c1_temp[1:-1].split()]
    c2_temp = df3["content 2"][i]
    c2 = [float(x) for x in c2_temp[1:-1].split()]
    ce = [c1, c2]
    x2.append(ce)
for i in range(Train):
    c1_temp = df4["content 1"][i]
    c1 = [float(x) for x in c1_temp[1:-1].split()]
    c2_temp = df4["content 2"][i]
    c2 = [float(x) for x in c2_temp[1:-1].split()]
    ce = [c1, c2]
    x2.append(ce)

y1 = np.ones(Train)
y2 = np.zeros(Train)
y2_train = np.concatenate((y1, y2))

In [9]:
### Define kernels

def srank_kernel(X, Y):
    """
    We create a custom stable rank kernel
    ref: JENS AGERBERG, Statistical Learning and Analysis on Homology-Based Features
    X,Y: stable rank array input data
    """
    t_union = np.union1d(X[0], Y[0])
    tY_diff = np.setdiff1d(Y[0], X[0]) #t in Y but not in X
    tX_diff = np.setdiff1d(X[0], Y[0]) #t in X but not in Y
    M1 = np.zeros(len(t_union))
    M2 = np.zeros(len(t_union))
    x_offset = 0
    y_offset = 0
    for i in range(len(t_union)):
        if t_union[i] <= X[0][-1] and t_union[i] == X[0][i-x_offset]:
            M1[i] = X[1][i-x_offset]
        else:
            x_offset = x_offset + 1
            M1[i] = X[1][i-x_offset]
            
        if t_union[i] <= Y[0][-1] and t_union[i] == Y[0][i-y_offset]:
            M2[i] = Y[1][i-y_offset]
        else:
            y_offset = y_offset + 1
            M2[i] = Y[1][i-y_offset]
    t_diff = np.ediff1d(t_union)
    return np.dot(np.multiply(M1[0:len(M1)-1], M2[0:len(M2)-1]), t_diff)

#this requires return a Gram matrix
def proxy_kernel(X,Y):
    gram_matrix = np.zeros((len(X), len(Y)))
    for i, x in enumerate(X):
        for j, y in enumerate(Y):
            gram_matrix[i, j] = srank_kernel(x, y)
    return gram_matrix

In [10]:
### Fit svm

svm = svm.SVC(kernel=proxy_kernel)
svm.fit(x2, y2_train)

SVC(kernel=<function proxy_kernel at 0x7fca78e12c10>)

## Calculate Classification Accuracy

In [14]:
x3 = []
x4 = []

for i in range(Train+1,100):
    c1_temp = df3["content 1"][i]
    c1 = [float(x) for x in c1_temp[1:-1].split()]
    c2_temp = df3["content 2"][i]
    c2 = [float(x) for x in c2_temp[1:-1].split()]
    ce = [c1, c2]
    x3.append(ce)
for i in range(Train+1,100):
    c1_temp = df4["content 1"][i]
    c1 = [float(x) for x in c1_temp[1:-1].split()]
    c2_temp = df4["content 2"][i]
    c2 = [float(x) for x in c2_temp[1:-1].split()]
    ce = [c1, c2]
    x4.append(ce)

In [18]:
Acc = (sum(svm.predict(x3))/Train + (1-sum(svm.predict(x4))/Train))/2

In [19]:
Acc

0.5125